# UTC - EDA

Greg Welliver   

In [1]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats
import re
from glob import glob, iglob
from datetime import datetime
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime, timedelta


# Seaborn settings
sns.set_context('notebook')
#sns.set_style("whitegrid")
#sns.set_style("white")
style_set = sns.set_style('whitegrid')
pd.options.display.float_format = '{:,}'.format
color_choice1 = 'plum'
color_choice2 = 'lightblue'
color_choice3 = 'lightgreen'
#ax.yaxis.set_major_formatter('{x:,.0f}')
#sns.set_context('notebook', font_scale = 4)

%matplotlib inline
import pandas as pd
#from mpl_toolkits.basemap import Basemap

import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx

from sklearn.feature_extraction.text import CountVectorizer

import json
from ydata_profiling import ProfileReport


## Load the data



In [2]:
# load data
df1 = pd.read_json('logins.json')
df2 = pd.read_json('ultimate_data_challenge.json')



In [3]:
df1.head()

,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21


profile = ProfileReport(df1)
profile

### Data quality issues:
- according to the profiling report, there are 868 duplicate rows (0.9%) - there is no need to remove these, as it is possible that two users logged in at the same time
- there are not any nulls

In [4]:
ids = df1["login_time"]
df1[ids.isin(ids[ids.duplicated()])].sort_values("login_time")

,login_time
347,1970-01-02 10:59:52
354,1970-01-02 10:59:52
427,1970-01-02 12:27:48
434,1970-01-02 12:27:48
503,1970-01-02 15:08:27
...,...
92439,1970-04-12 17:12:21
92984,1970-04-13 12:23:46
92986,1970-04-13 12:23:46
93021,1970-04-13 13:44:18


In [5]:
df1.isna().value_counts()

login_time
False         93142
dtype: int64

In [6]:
df1.dtypes

login_time    datetime64[ns]
dtype: object

In [7]:
df1.login_time.max()

Timestamp('1970-04-13 18:57:38')

In [8]:
start = df1.login_time.min()
end = df1.login_time.max()

In [9]:
delta = 15
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

dts = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in 
       datetime_range(datetime(2016, 9, 1, 7), datetime(2016, 9, 1, 9+12), 
       timedelta(minutes=15))]

In [10]:
dts = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in 
       datetime_range(df1.login_time.min(), df1.login_time.max(), 
       timedelta(minutes=15))]

In [11]:
dts = pd.DataFrame(dts)

In [12]:
dts

,0
0,1970-01-01 20:12:16
1,1970-01-01 20:27:16
2,1970-01-01 20:42:16
3,1970-01-01 20:57:16
4,1970-01-01 21:12:16
...,...
9783,1970-04-13 17:57:16
9784,1970-04-13 18:12:16
9785,1970-04-13 18:27:16
9786,1970-04-13 18:42:16


In [13]:
df3 = pd.DataFrame()

In [14]:
df3['begin_time'] = dts

In [15]:
#new_df['begin_times2'] = pd.to_datetime(new_df['begin_times'])
df3['begin_time'] = pd.to_datetime(df3['begin_time'])

In [16]:
df3['begin_time']

0      1970-01-01 20:12:16
1      1970-01-01 20:27:16
2      1970-01-01 20:42:16
3      1970-01-01 20:57:16
4      1970-01-01 21:12:16
               ...        
9783   1970-04-13 17:57:16
9784   1970-04-13 18:12:16
9785   1970-04-13 18:27:16
9786   1970-04-13 18:42:16
9787   1970-04-13 18:57:16
Name: begin_time, Length: 9788, dtype: datetime64[ns]

In [17]:
df3['duration'] = 15

In [18]:
df3['end_time'] = df3['begin_time'] + pd.to_timedelta(df3['duration'], unit='m')
df3

,begin_time,duration,end_time
0,1970-01-01 20:12:16,15,1970-01-01 20:27:16
1,1970-01-01 20:27:16,15,1970-01-01 20:42:16
2,1970-01-01 20:42:16,15,1970-01-01 20:57:16
3,1970-01-01 20:57:16,15,1970-01-01 21:12:16
4,1970-01-01 21:12:16,15,1970-01-01 21:27:16
...,...,...,...
9783,1970-04-13 17:57:16,15,1970-04-13 18:12:16
9784,1970-04-13 18:12:16,15,1970-04-13 18:27:16
9785,1970-04-13 18:27:16,15,1970-04-13 18:42:16
9786,1970-04-13 18:42:16,15,1970-04-13 18:57:16


In [19]:
#df3 = df3.drop(['duration'], axis=1, inplace=True)
df3 = df3.drop(['duration'], axis=1)

In [20]:
df3

,begin_time,end_time
0,1970-01-01 20:12:16,1970-01-01 20:27:16
1,1970-01-01 20:27:16,1970-01-01 20:42:16
2,1970-01-01 20:42:16,1970-01-01 20:57:16
3,1970-01-01 20:57:16,1970-01-01 21:12:16
4,1970-01-01 21:12:16,1970-01-01 21:27:16
...,...,...
9783,1970-04-13 17:57:16,1970-04-13 18:12:16
9784,1970-04-13 18:12:16,1970-04-13 18:27:16
9785,1970-04-13 18:27:16,1970-04-13 18:42:16
9786,1970-04-13 18:42:16,1970-04-13 18:57:16


In [21]:
df1

,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21
...,...
93137,1970-04-13 18:50:19
93138,1970-04-13 18:43:56
93139,1970-04-13 18:54:02
93140,1970-04-13 18:57:38


In [22]:
df3['login_count'] = 0

In [23]:
# for i in df3['login_count'][:5]:
#     #print(df3['login_count'][i])
#     for x in df1['login_time']:
#         print(df1['login_time'][:5])
#         #print(df1['login_time'][x])

for i in df1['login_count']

In [24]:
# # converting index to datetime index
# df1['login_time'].index = pd.to_datetime(df1['login_time'])
  
# # Changing start time for each hour, by default start time is at 0th minute
# df1['login_time'].resample('W',  loffset='30Min30s').login_time.sum().head(2)
# df1['login_time'].resample('W', loffset='30Min30s').login_time.sum().head(2)
  
# # we can also aggregate it will show quantity added in each week
# # as well as the total amount added in each week
# df1['login_time'].resample('W', loffset='30Min30s').agg(
#     {'login_time': 'sum'}).head(5)

In [25]:
#df1['login_time'].index = pd.to_datetime(df1['login_time'])

for i in df3['login_count'][:10]:
    df3['login_count'][i] = 0
    for x in df1["login_time"]:
        if df1["login_time"][x] >= df3['begin_time'][i] and df1["login_time"][x] < df3['end_time'][i]:
            df3['login_count'][i] += 1


In [28]:
df1["login_time"].rollup(sum, 120)

AttributeError: 'Series' object has no attribute 'rollup'

In [ ]:
df1["login_time"]

In [29]:
col_list = df1["login_time"].values.tolist()
print(col_list)

[72798000000000, 72970000000000, 72997000000000, 72996000000000, 73581000000000, 73301000000000, 72736000000000, 74147000000000, 74138000000000, 74872000000000, 73565000000000, 73863000000000, 74086000000000, 74194000000000, 74365000000000, 74437000000000, 74098000000000, 74639000000000, 74866000000000, 74794000000000, 75367000000000, 74952000000000, 75546000000000, 75599000000000, 76501000000000, 76164000000000, 76764000000000, 76853000000000, 77359000000000, 79085000000000, 78711000000000, 79180000000000, 79714000000000, 79371000000000, 79112000000000, 79351000000000, 79532000000000, 79366000000000, 79828000000000, 79918000000000, 79265000000000, 80350000000000, 80214000000000, 80252000000000, 79925000000000, 80583000000000, 80534000000000, 80709000000000, 80644000000000, 80399000000000, 81092000000000, 83171000000000, 82556000000000, 83229000000000, 83516000000000, 83075000000000, 83068000000000, 83571000000000, 83285000000000, 83497000000000, 83371000000000, 82675000000000, 8364000

In [30]:
array = df1["login_time"].to_numpy()

In [31]:
array

array(['1970-01-01T20:13:18.000000000', '1970-01-01T20:16:10.000000000',
       '1970-01-01T20:16:37.000000000', ...,
       '1970-04-13T18:54:02.000000000', '1970-04-13T18:57:38.000000000',
       '1970-04-13T18:54:23.000000000'], dtype='datetime64[ns]')

In [33]:
col_list.rollup(sum, 120)

AttributeError: 'list' object has no attribute 'rollup'

In [35]:
df1["login_time"].resample("15T").agg({'count':'sum'})

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [27]:
for i in df3['login_count'][:10]:
    df3['login_count'][i] = 0
    for x in df1["login_time"]:
        if df1["login_time"][x] >= df3['begin_time'][i]:
            df3['login_count'][i] += 1


/var/folders/rc/20gvvbw10mg1xrcm82p5g4tm0000gn/T/ipykernel_10999/470418571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['login_count'][i] = 0


KeyError: Timestamp('1970-01-01 20:13:18')

In [ ]:
df3

In [ ]:
#new_df.end_times2 = new_df.apply(lambda x: x.begin_times2 + pd.Timedelta(Second(new_df.duration)) ,axis = 1)

### brainstorm to create intervals:

- create a dataframe with 15 minute intervals --> start time is 15 minute intervals, and end time is beginning of interval + X number of seconds
- column with begin time
- column with end time
- loop through login file and count how many logins are between those two times